In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path) 

In [2]:
%load_ext autoreload
%autoreload 2

# Libs

In [3]:
import torch
from torch.utils.data import DataLoader, RandomSampler
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import os
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm
import gc

import src

In [ ]:
!apt-get

In [25]:
!nvidia-smi

Thu Jun 18 17:07:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   54C    P0    57W / 149W |   1745MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
torch.cuda.is_available()

True

# Train funciton

In [23]:
def train(model, train_loader,
          test_loader,
          loss,
          optimizer,
          scheduler,
          tb_path=".",
          epochs = 1,
          init_epoch=0):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Device: {}".format(device))
    print('*****************')
    
    writer = SummaryWriter(tb_path)
    
    model = model.to(device)

    train_loss_history = []
    test_loss_history = []
  
    for epoch in range(init_epoch, init_epoch+epochs):
        print(f"Epoch {epoch+1}:")
        
        train_mean_loss = 0
        
        pbar = tqdm(total=len(train_loader))
        
        
        model = model.train()
        for i, (swn, s) in enumerate(train_loader):
            optimizer.zero_grad()
            model.zero_grad()


            swn = swn.to(device)
            s = s.to(device)
            
            cs = model.forward(swn)
            loss_value = loss(cs, swn, s)
            
            
            if torch.isnan(loss_value):
                print("Error on {}".format(i))
                del cs
                del swn
                del s
                del loss_value
                gc.collect()
                torch.cuda.empty_cache()
                continue
                #return

            loss_value.backward()
            train_mean_loss += loss_value.data.cpu()
            
            optimizer.step()
            
            pbar.update(1)
            
            pbar.set_description("Loss \"{}\"".format(loss_value.data.cpu()))
        
            gc.collect()
            torch.cuda.empty_cache()
            del cs
            del swn
            del s
            del loss_value
            gc.collect()
            torch.cuda.empty_cache()
        
        scheduler.step()
        torch.cuda.empty_cache()
        pbar.close()
        train_mean_loss /= len(train_loader)
        train_loss_history.append(train_mean_loss)
        print(f"Mean train loss: {train_mean_loss}:")

        example_audio = True
        
        test_mean_loss = 0
        count = 0
        model = model.eval()
        for (swn, s) in train_loader:
            

            swn = swn.to(device)
            s = s.to(device)

            cs = model.forward(swn)
            loss_value = loss(cs, swn, s)
            count += 1
            
            if example_audio:
                example_audio = False
                writer.add_audio('Audio/clean', s[0,:].data.cpu(), sample_rate=16000)
                writer.add_audio('Audio/with noise', swn[0,:].data.cpu(), sample_rate=16000)
                writer.add_audio('Audio/predict', cs[0,:].data.cpu(), sample_rate=16000)
            test_mean_loss += loss_value.data.cpu()
            
            gc.collect()
            torch.cuda.empty_cache()
            del cs
            del swn
            del s
            del loss_value
            

        test_mean_loss /= count
        test_loss_history.append(test_mean_loss)
        print(f"Mean test loss: {test_mean_loss}:")


        
        writer.add_scalar('Loss/train', train_mean_loss, epoch)
        writer.add_scalar('Loss/test', test_mean_loss, epoch)
        print('---------------')
    return train_loss_history, test_loss_history

# Experiment config

In [7]:
train_speech_sources = list(Path('../data/cleaned/LIBRISPEECH/LibriSpeech/train-clean-100/').glob('*'))[:50]
train_speech_files = np.concatenate([
    list(path.glob('**/*.flac')) for path in train_speech_sources
])

test_speech_sources = list(Path('../data/cleaned/LIBRISPEECH/LibriSpeech/train-clean-100/').glob('*'))[50:55]
test_speech_files = np.concatenate([
    list(path.glob('**/*.flac')) for path in test_speech_sources
])

train_noise_sources = list(Path('../data/cleaned/DEMAND').glob('*'))[:8]
train_noise_files = np.concatenate([
    list(path.glob('**/*.wav')) for path in train_noise_sources
])

test_noise_sources = list(Path('../data/cleaned/DEMAND').glob('*'))[8:10]
test_noise_files = np.concatenate([
    list(path.glob('**/*.wav')) for path in test_noise_sources
])


In [15]:
NAME = "Experiment 1"
CONFIG = {
    "name": NAME, 
    "snr": [15, 10, 5, 0],
    "results_path": "../results/{}".format(NAME),
    "tensorboard_path": "../results/tb/{}".format(NAME),
    "train_samples": 20,
    "test_samples": 10,
    "train_speech_batch_size": 10,
    "test_speech_batch_size": 1,
    "shuffle": True,
    "train_speech": train_speech_files,
    "test_speech": test_speech_files,
    "train_noise": train_noise_files,
    "test_noise": test_noise_files,
}
CONFIG

{'name': 'Experiment 1',
 'snr': [15, 10, 5, 0],
 'results_path': '../results/Experiment 1',
 'tensorboard_path': '../results/tb/Experiment 1',
 'train_samples': 20,
 'test_samples': 10,
 'train_speech_batch_size': 10,
 'test_speech_batch_size': 1,
 'shuffle': True,
 'train_speech': array([PosixPath('../data/cleaned/LIBRISPEECH/LibriSpeech/train-clean-100/6019/3185/6019-3185-0089.flac'),
        PosixPath('../data/cleaned/LIBRISPEECH/LibriSpeech/train-clean-100/6019/3185/6019-3185-0010.flac'),
        PosixPath('../data/cleaned/LIBRISPEECH/LibriSpeech/train-clean-100/6019/3185/6019-3185-0098.flac'),
        ...,
        PosixPath('../data/cleaned/LIBRISPEECH/LibriSpeech/train-clean-100/1926/147979/1926-147979-0037.flac'),
        PosixPath('../data/cleaned/LIBRISPEECH/LibriSpeech/train-clean-100/1926/147979/1926-147979-0028.flac'),
        PosixPath('../data/cleaned/LIBRISPEECH/LibriSpeech/train-clean-100/1926/147979/1926-147979-0046.flac')],
       dtype=object),
 'test_speech': array

# Data

In [16]:
seed = 3
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Train

In [17]:
train_dataset = src.SpeechWithNoiseDataset(
    CONFIG["train_speech"],
    CONFIG["train_noise"],
    speech_batch_size=CONFIG["train_speech_batch_size"],
    noise_by_speech=4,
    max_len=None if CONFIG["shuffle"] else CONFIG["train_samples"],
    snr=CONFIG["snr"]
)
if CONFIG["shuffle"]:
    sampler = RandomSampler(
        train_dataset,
        replacement=True, 
        num_samples=CONFIG["train_samples"]
    )
else:
    sampler = None
train_loader = DataLoader(
    train_dataset,
    batch_size = None,
    sampler = sampler,
    num_workers = 3
)

# Test

In [18]:
test_dataset = src.SpeechWithNoiseDataset(
    CONFIG["test_speech"],
    CONFIG["test_noise"],
    speech_batch_size=CONFIG["test_speech_batch_size"],
    noise_by_speech=4,
    max_len=None if CONFIG["shuffle"] else CONFIG["test_samples"],
    snr=CONFIG["snr"]
)
if CONFIG["shuffle"]:
    sampler = RandomSampler(
        test_dataset,
        replacement=True, 
        num_samples=CONFIG["test_samples"]
    )
else:
    sampler = None
test_loader = DataLoader(
    test_dataset,
    batch_size = None,
    sampler = sampler,
    num_workers = 3
)

# Model

In [12]:
conv = src.DCUnet10(iscomplex = True)
model = src.EnhModel(conv, 64*16, 16*16)

In [35]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
def loss(cs, swn, s, **kvargs):
    return src.wsdr_loss(
        cs,
        swn[:, 0:cs.shape[1]],
        s[:, 0:cs.shape[1]],
        **kvargs
    )
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [37]:
train(
    model,
    train_loader,
    test_loader,
    loss,
    optimizer,
    scheduler,
    tb_path=CONFIG["tensorboard_path"],
    init_epoch=5,
    epochs = 100
)

Device: cuda
*****************
Epoch 6:



Mean train loss: -0.12390901893377304:
Mean test loss: -0.12905803322792053:
---------------
Epoch 7:



Mean train loss: -0.11189625412225723:
Mean test loss: -0.10447792708873749:
---------------
Epoch 8:



Mean train loss: -0.1353749930858612:
Mean test loss: -0.12434504181146622:
---------------
Epoch 9:



Mean train loss: -0.11731763929128647:
Mean test loss: -0.11462334543466568:
---------------
Epoch 10:


...


Mean train loss: -0.11449827998876572:
Mean test loss: -0.16470867395401:
---------------
Epoch 21:



Mean train loss: -0.13901129364967346:
Mean test loss: -0.143695667386055:
---------------
Epoch 22:



Mean train loss: -0.13821814954280853:
Mean test loss: -0.12923437356948853:
---------------
Epoch 23:



Mean train loss: -0.1350826919078827:
Mean test loss: -0.143620103597641:
---------------
Epoch 24:


KeyboardInterrupt: 

In [38]:
os.makedirs(CONFIG["results_path"], exist_ok = True) 
torch.save(model, os.path.join(CONFIG["results_path"], "mode.pth"))

/opt/conda/envs/vkinternship/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type EnhModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/envs/vkinternship/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DCUnet10. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/envs/vkinternship/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/envs/vkinternship/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.


In [39]:
os.makedirs(CONFIG["results_path"], exist_ok = True) 
torch.save(model.state_dict(), os.path.join(CONFIG["results_path"], "mode_state.pth"))

# Metrics

In [44]:
from pypesq import pesq

In [53]:
N = 100
mean_pesq = 0
model = model.cpu()
for i in tqdm(range(N)):
    swn, s = test_dataset[i]
    cs = model(swn[[0], :])
    mean_pesq += pesq(s[0,:cs.shape[1]].data.numpy(), cs[0,:].data.numpy(), fs=16000)
mean_pesq /= N

In [54]:
mean_pesq

2.313958193063736